# Purpose

Use this notebook to test the new data-loader (GCS) & configs that we'll be using in kubeflow.

Why? For inference (getting embeddings) it might be better to read from GCS than from SQL.

# Imports & Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import seaborn as sns

from tqdm import tqdm

import subclu
# from subclu.utils import set_working_directory
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric
)

# new modules to test:
from google.cloud import bigquery

from subclu.i18n_topic_model_batch.subclu2.utils.data_loaders_gcs import (
    LoadSubredditsGCS
)


print_lib_versions([bigquery, np, pd, plotly, sns, subclu])

python		v 3.7.10
===
google.cloud.bigquery	v: 2.20.0
numpy		v: 1.18.5
pandas		v: 1.2.5
plotly		v: 4.14.3
seaborn		v: 0.11.1
subclu		v: 0.5.0


In [3]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()
logging.info('loggging ready')

03:39:54 | INFO | "loggging ready"


# Load data with new class

Load data from a test folder where we have 2 parquet files. This way we can test slicing & sampling.

## Cache files

### All files, read as 1 pandas df

In [18]:
%%time

subs = LoadSubredditsGCS(
    bucket_name='gazette-models-temp',
    gcs_path='i18n_topic_model_batch/runs/20220412/subreddits/text',
    local_cache_path="/home/jupyter/subreddit_clustering_i18n/data/local_cache/",
    columns=['subreddit_id', 'subreddit_name', 'subreddit_name_title_related_subs_and_clean_descriptions'],
    col_unique_check='subreddit_id',
    df_format='pandas',
    n_sample_files=None,
    n_files_slice_start=None,
    n_files_slice_end=None,
    unique_check=False,
    verbose= True,
)
subs.local_cache()

print(f"{len(subs.local_files_)} <- Local files")
print(f"{len(subs.local_parquet_files_)} <- Local parquet files")
assert 2 == len(subs.local_files_)

df_ = subs.read_as_one_df()
print(df_.shape)

08:40:43 | INFO | "  Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/gazette-models-temp/i18n_topic_model_batch/runs/20220412/subreddits/text"
08:40:43 | INFO | "  2 <- Files matching prefix"
08:40:43 | INFO | "  2 <- Files to check"
08:40:43 | INFO | "    000000000000.parquet <- File already exists, not downloading"
08:40:43 | INFO | "    000000000001_test.parquet <- File already exists, not downloading"
08:40:43 | INFO | "  Files already cached: 2"
08:40:43 | INFO | "0:00:01.398668  <- Downloading files elapsed time"
08:40:43 | INFO | "  Files already downloaded."
08:40:43 | INFO | "  df format: pandas"


2 <- Local files
2 <- Local parquet files
(159874, 3)
CPU times: user 385 ms, sys: 307 ms, total: 693 ms
Wall time: 1.9 s


In [19]:
df_.head()

,subreddit_id,subreddit_name,subreddit_name_title_related_subs_and_clean_descriptions
0,t5_2sxhs,place,"place. r/place. \nplace\nplace\n\nSome have visited a canvas before. A place where togetherness created more. Now in numbers far greater, taking more space, It falls upon you to create a better place.\n\nThere is an empty canvas.\n\nYou..."
1,t5_2qh1i,askreddit,"AskReddit. r/AskReddit. \nAsk Reddit...\naskreddit, AskReddit\n\nAskReddit is the place to ask and answer thought provoking questions.\n\nSERIOUS askreddit true Serious \n\nRules AskReddit wikiwiki rules:\n1. You must post a clear and d..."
2,t5_2qhsa,interestingasfuck,"interestingasfuck. r/interestingasfuck. \nInteresting As Fuck\ninterestingasfuck, TodayILearned, notinteresting, mildlyinteresting, offbeat, oddlysatisfying, damnthatsinteresting, Unexpected, wtf\n\nFor anything that is InterestingAsFuc..."
3,t5_2y77d,antiwork,"antiwork. r/antiwork. \nAntiwork: Unemployment for all, not just the rich!\nantiwork, antitrampo, AntiTaff, antiarbeit, antiworkItaly, tegenwerken, antiwork_slovenija, Antiwork_UK, Anarchism, Anarchy101, IWW, LateStageCapitalism, lostge..."
4,t5_2qh13,worldnews,"worldnews. r/worldnews. \nWorld News\nNews, PoliticalDiscussion, WorldEvents, GeoPolitics, IntheNews, GlobalTalk, Breakingnews, Business, Economics, Environment, History, HumanRights, Features, UpliftingNews, NewsOfTheWeird, FakeNews, I..."


### All files, read as 1 `dask` df

In [20]:
%%time

subs = LoadSubredditsGCS(
    bucket_name='gazette-models-temp',
    gcs_path='i18n_topic_model_batch/runs/20220412/subreddits/text',
    local_cache_path="/home/jupyter/subreddit_clustering_i18n/data/local_cache/",
    columns=['subreddit_id', 'subreddit_name', 'subreddit_name_title_related_subs_and_clean_descriptions'],
    col_unique_check='subreddit_id',
    df_format='dask',
    n_sample_files=None,
    n_files_slice_start=None,
    n_files_slice_end=None,
    unique_check=False,
    verbose= True,
)
subs.local_cache()

print(f"{len(subs.local_files_)} <- Local files")
print(f"{len(subs.local_parquet_files_)} <- Local parquet files")
assert 2 == len(subs.local_files_)

df_ = subs.read_as_one_df()
print(df_.shape)

08:40:49 | INFO | "  Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/gazette-models-temp/i18n_topic_model_batch/runs/20220412/subreddits/text"
08:40:49 | INFO | "  2 <- Files matching prefix"
08:40:49 | INFO | "  2 <- Files to check"
08:40:49 | INFO | "    000000000000.parquet <- File already exists, not downloading"
08:40:49 | INFO | "    000000000001_test.parquet <- File already exists, not downloading"
08:40:49 | INFO | "  Files already cached: 2"
08:40:49 | INFO | "0:00:01.388091  <- Downloading files elapsed time"
08:40:49 | INFO | "  Files already downloaded."
08:40:49 | INFO | "  df format: dask"


2 <- Local files
2 <- Local parquet files
(Delayed('int-b056f9c0-d5ba-4540-b879-5b51a63c2e51'), 3)
CPU times: user 49.1 ms, sys: 49.1 ms, total: 98.2 ms
Wall time: 1.41 s


In [21]:
df_.tail()

,subreddit_id,subreddit_name,subreddit_name_title_related_subs_and_clean_descriptions
79932,t5_s3j8c,antianimearmy,"AntiAnimeArmy. r/AntiAnimeArmy. \nJoin the Army Today!\nAnimeAddictsAnonymous, NoAnimePolice, AnimeHate\n\nThe Anti Anime Army is to destroy the evil that is Anime.\n\nSeeking help with your addiction?\ntry our spin off subreddit:\nAnim..."
79933,t5_3stsd4,poorlymadepolicememes,poorlymadepolicememes. r/poorlymadepolicememes. \npoorlymadepolicememes\nThe sub about the very poorly made memer and podcaster\n\nFeel free to discuss the podcast episodes and memes\n\nA bunch of police memes that aren’t really that gr...
79934,t5_48te5o,svampedyrkning,"Svampedyrkning. r/Svampedyrkning. \nSvampedyrkning\nEt dansk subreddit dedikeret til svampedyrkning. Her kan man diskutere dyrkning af gourmet , psilocybin og medicinsvampe, med henblik på at hjælpe hinanden, dele sine erfaringer og udv..."
79935,t5_4ua7kp,meditieren,"meditieren. r/meditieren. \nMeditieren - Entspannung von Kopf bis Fuß.\nmeditieren\n\nWillkommen auf meditieren! Dieser Sub dient dem Austausch von Erfahrungen, Geschichten und Anleitungen in Bezug auf die Praxis der Meditation.\n\nWILK..."
79936,t5_4cefbk,handball_de,handball_de. r/handball_de. \nHandball\nFür alles rund um den Profi und Amateurhandball!


### All files, yield each file as separate df

In [23]:
%%time

subs = LoadSubredditsGCS(
    bucket_name='gazette-models-temp',
    gcs_path='i18n_topic_model_batch/runs/20220412/subreddits/text',
    local_cache_path="/home/jupyter/subreddit_clustering_i18n/data/local_cache/",
    columns=['subreddit_id', 'subreddit_name', 'subreddit_name_title_related_subs_and_clean_descriptions'],
    col_unique_check='subreddit_id',
    df_format='pandas',
    n_sample_files=None,
    n_files_slice_start=None,
    n_files_slice_end=None,
    unique_check=False,
    verbose= True,
)
subs.local_cache()
for f_, df_y_ in tqdm(subs.yield_files_and_dfs(), total=subs.n_local_parquet_files_):
    print(f_.name)
    print(df_y_.shape)

print(f"{len(subs.local_files_)} <- Local files")
print(f"{len(subs.local_parquet_files_)} <- Local parquet files")
assert 2 == len(subs.local_files_)

08:44:40 | INFO | "  Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/gazette-models-temp/i18n_topic_model_batch/runs/20220412/subreddits/text"
08:44:40 | INFO | "  4 <- Files matching prefix"
08:44:40 | INFO | "  4 <- Files to check"
08:44:40 | INFO | "    000000000000.parquet <- File already exists, not downloading"
08:44:40 | INFO | "    000000000001_test.parquet <- File already exists, not downloading"
08:44:40 | INFO | "  Files already cached: 2"
08:44:40 | INFO | "0:00:01.392648  <- Downloading files elapsed time"
100%|██████████| 2/2 [00:00<00:00,  5.03it/s]

000000000000.parquet
(79937, 3)
000000000001_test.parquet
(79937, 3)
2 <- Local files
2 <- Local parquet files
CPU times: user 298 ms, sys: 209 ms, total: 507 ms
Wall time: 1.79 s


### Sample file (1st file)

In [10]:
%%time

subs = LoadSubredditsGCS(
    bucket_name='gazette-models-temp',
    gcs_path='i18n_topic_model_batch/runs/20220412/subreddits/text',
    local_cache_path="/home/jupyter/subreddit_clustering_i18n/data/local_cache/",
    columns=['subreddit_id', 'subreddit_name', 'subreddit_description'],
    col_unique_check='subreddit_id',
    df_format='pandas',
    n_sample_files=1,
    n_files_slice_start=None,
    n_files_slice_end=None,
    unique_check=True,
    verbose= True,
)
df_ = subs.read_as_one_df()
print(df_.shape)

print(f"{len(subs.local_files_)} <- Local files")
print(f"{len(subs.local_parquet_files_)} <- Local parquet files")

assert(1 == len(subs.local_files_)), "Expected to sample only 1 file"

03:40:01 | INFO | "  Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/gazette-models-temp/i18n_topic_model_batch/runs/20220412/subreddits/text"
03:40:01 | INFO | "  1 <- Files matching prefix"
03:40:01 | INFO | "  1 <- Files to check"
03:40:01 | INFO | "    000000000000.parquet <- File already exists, not downloading"
03:40:01 | INFO | "  Files already cached: 1"
03:40:01 | INFO | "0:00:01.394998  <- Downloading files elapsed time"
03:40:01 | INFO | "  df format: pandas"
03:40:01 | INFO | "  Checking ID uniqueness..."


(79937, 3)
1 <- Local files
1 <- Local parquet files
CPU times: user 174 ms, sys: 77.9 ms, total: 252 ms
Wall time: 1.57 s


### Slice -- last file

In [11]:
%%time

subs = LoadSubredditsGCS(
    bucket_name='gazette-models-temp',
    gcs_path='i18n_topic_model_batch/runs/20220412/subreddits/text',
    local_cache_path="/home/jupyter/subreddit_clustering_i18n/data/local_cache/",
    columns=['subreddit_id', 'subreddit_name', 'subreddit_description'],
    col_unique_check='subreddit_id',
    df_format='pandas',
    n_sample_files=None,
    n_files_slice_start=-1,
    n_files_slice_end=None,
    unique_check=True,
    verbose= True,
)
df_ = subs.read_as_one_df()
print(df_.shape)

print(f"{len(subs.local_files_)} <- Local files")
print(f"{len(subs.local_parquet_files_)} <- Local parquet files")

assert(1 == len(subs.local_files_)), "Expected slice with only 1 file"

03:40:03 | INFO | "  Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/gazette-models-temp/i18n_topic_model_batch/runs/20220412/subreddits/text"
03:40:03 | INFO | "  2 <- Files matching prefix"
03:40:03 | INFO | "  1 <- Files to check"
03:40:03 | INFO | "    000000000001_test.parquet <- File already exists, not downloading"
03:40:03 | INFO | "  Files already cached: 1"
03:40:03 | INFO | "0:00:01.361892  <- Downloading files elapsed time"
03:40:03 | INFO | "  df format: pandas"
03:40:03 | INFO | "  Checking ID uniqueness..."


(79937, 3)
1 <- Local files
1 <- Local parquet files
CPU times: user 184 ms, sys: 136 ms, total: 321 ms
Wall time: 1.6 s


In [12]:
%%time

subs = LoadSubredditsGCS(
    bucket_name='gazette-models-temp',
    gcs_path='i18n_topic_model_batch/runs/20220412/subreddits/text',
    local_cache_path="/home/jupyter/subreddit_clustering_i18n/data/local_cache/",
    columns=['subreddit_id', 'subreddit_name', 'subreddit_description'],
    col_unique_check='subreddit_id',
    df_format='pandas',
  
    n_sample_files=None,
    n_files_slice_start=1,
    n_files_slice_end=2,
    unique_check=True,
    verbose= True,
)
df_ = subs.read_as_one_df()
print(df_.shape)

print(f"{len(subs.local_files_)} <- Local files")
print(f"{len(subs.local_parquet_files_)} <- Local parquet files")

assert(1 == len(subs.local_files_)), "Expected slice with only 1 file"

03:40:04 | INFO | "  Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/gazette-models-temp/i18n_topic_model_batch/runs/20220412/subreddits/text"
03:40:04 | INFO | "  2 <- Files matching prefix"
03:40:04 | INFO | "  1 <- Files to check"
03:40:04 | INFO | "    000000000001_test.parquet <- File already exists, not downloading"
03:40:04 | INFO | "  Files already cached: 1"
03:40:04 | INFO | "0:00:01.336908  <- Downloading files elapsed time"
03:40:04 | INFO | "  df format: pandas"
03:40:05 | INFO | "  Checking ID uniqueness..."


(79937, 3)
1 <- Local files
1 <- Local parquet files
CPU times: user 184 ms, sys: 141 ms, total: 326 ms
Wall time: 1.59 s


### Slice first file

In [13]:
%%time

subs = LoadSubredditsGCS(
    bucket_name='gazette-models-temp',
    gcs_path='i18n_topic_model_batch/runs/20220412/subreddits/text',
    local_cache_path="/home/jupyter/subreddit_clustering_i18n/data/local_cache/",
    columns=['subreddit_id', 'subreddit_name', 'subreddit_description'],
    col_unique_check='subreddit_id',
    df_format='pandas',
    n_sample_files=None,
    n_files_slice_start=None,
    n_files_slice_end=1,
    unique_check=True,
    verbose= True,
)
df_ = subs.read_as_one_df()
print(df_.shape)

print(f"{len(subs.local_files_)} <- Local files")
print(f"{len(subs.local_parquet_files_)} <- Local parquet files")

assert(1 == len(subs.local_files_)), "Expected slice with 1 file"

03:40:06 | INFO | "  Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/gazette-models-temp/i18n_topic_model_batch/runs/20220412/subreddits/text"
03:40:06 | INFO | "  2 <- Files matching prefix"
03:40:06 | INFO | "  1 <- Files to check"
03:40:06 | INFO | "    000000000000.parquet <- File already exists, not downloading"
03:40:06 | INFO | "  Files already cached: 1"
03:40:06 | INFO | "0:00:01.384844  <- Downloading files elapsed time"
03:40:06 | INFO | "  df format: pandas"
03:40:06 | INFO | "  Checking ID uniqueness..."


(79937, 3)
1 <- Local files
1 <- Local parquet files
CPU times: user 191 ms, sys: 126 ms, total: 317 ms
Wall time: 1.62 s


### Slice first 2 files

In [14]:
%%time

subs = LoadSubredditsGCS(
    bucket_name='gazette-models-temp',
    gcs_path='i18n_topic_model_batch/runs/20220412/subreddits/text',
    local_cache_path="/home/jupyter/subreddit_clustering_i18n/data/local_cache/",
    columns=['subreddit_id', 'subreddit_name', 'subreddit_description'],
    col_unique_check='subreddit_id',
    df_format='pandas',
    n_sample_files=None,
    n_files_slice_start=None,
    n_files_slice_end=2,
    unique_check=False,
    verbose= True,
)
df_ = subs.read_as_one_df()
print(df_.shape)

print(f"{len(subs.local_files_)} <- Local files")
print(f"{len(subs.local_parquet_files_)} <- Local parquet files")

assert(2 == len(subs.local_files_)), "Expected 2 files"

03:40:08 | INFO | "  Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/gazette-models-temp/i18n_topic_model_batch/runs/20220412/subreddits/text"
03:40:08 | INFO | "  2 <- Files matching prefix"
03:40:08 | INFO | "  2 <- Files to check"
03:40:08 | INFO | "    000000000000.parquet <- File already exists, not downloading"
03:40:08 | INFO | "    000000000001_test.parquet <- File already exists, not downloading"
03:40:08 | INFO | "  Files already cached: 2"
03:40:08 | INFO | "0:00:01.365880  <- Downloading files elapsed time"
03:40:08 | INFO | "  df format: pandas"


(159874, 3)
2 <- Local files
2 <- Local parquet files
CPU times: user 321 ms, sys: 240 ms, total: 561 ms
Wall time: 1.77 s
